In [6]:
pip install medpy

     |████████████████████████████████| 151 kB 552 kB/s eta 0:00:01
     |████████████████████████████████| 48.4 MB 20 kB/s  eta 0:00:01
  Created wheel for medpy: filename=MedPy-0.4.0-py3-none-any.whl size=214963 sha256=8776bd0d0baff31c60d8629823b98080aa0fdee3b77dfd9bfd9bb50a149887c2
  Stored in directory: /home/ashwani/.cache/pip/wheels/41/46/a2/7c585b78f216a3dd8723dbab5f439822fa5dfbff563757a49e
Successfully built medpy
Note: you may need to restart the kernel to use updated packages.


In [39]:
import numpy as np
from numpy import load
import utils
import os
import copy
from skimage import img_as_float

import multiprocessing
from multiprocessing import Pool
# import the builtin time module
import time

In [40]:
PATH = "../test_files/output/"

In [41]:
t1ce_data = np.load(PATH + "t1.npz")['arr_0']

In [42]:
t1ce_data.shape

(152, 240, 240, 1)

In [7]:
def rician_wholedataset1(dataset, BaseVal, Scale, NumSamples):
    data = copy.deepcopy(dataset)
    for i in range(data.shape[0]):
        data[i, :, :, :] *= 255.0/data[i, :, :, :].max()
        noise = np.random.normal(scale=Scale, size=(
            NumSamples, 2)) + [[BaseVal, 0]]
        noise = np.linalg.norm(noise, axis=1)
        data[i, :, :, :] = img_as_float(
            data[i, :, :, :]) + noise.reshape(data[i, :, :, :].shape)
        data[i, :, :, :] *= 255.0/data[i, :, :, :].max()
        data[i] = data[i] / 255.0
    return data

In [13]:
def rician_wholedataset2(dataset, BaseVal, Scale, NumSamples):
    data = copy.deepcopy(dataset)
    noise = np.random.normal(scale=Scale, size=(NumSamples, 2)) + [[BaseVal, 0]]
    noise = np.linalg.norm(noise, axis=1)
    for i in range(data.shape[0]):
        data[i, :, :, :] *= 255.0/data[i, :, :, :].max()
        data[i, :, :, :] = img_as_float(data[i, :, :, :]) + noise.reshape(data[i, :, :, :].shape)
        data[i, :, :, :] *= 255.0/data[i, :, :, :].max()
        data[i] = data[i] / 255.0
    return data

In [16]:
def rician_wholedataset_inplace(data, BaseVal, Scale, NumSamples):
    # having it inplace helps a lot
    # having this noise thing outside outside helps w speed
    noise = np.random.normal(scale=Scale, size=(NumSamples, 2)) + [[BaseVal, 0]]
    noise = np.linalg.norm(noise, axis=1)
    for i in range(data.shape[0]):
        print(data[i, :, :, :].max())
        data[i, :, :, :] *= 255.0/data[i, :, :, :].max()
        data[i, :, :, :] = img_as_float(data[i, :, :, :]) + noise.reshape(data[i, :, :, :].shape)
        print(data[i, :, :, :].max())
        data[i, :, :, :] *= 255.0/data[i, :, :, :].max()
        data[i] = data[i] / 255.0
    return data

In [8]:
def ricianprocess(img, noise):
    1img *= 255.0/img.max()
    img = img_as_float(img) + noise.reshape(img.shape)
    img *= 255.0/img.max()
    img = img / 255.0
    return img

In [9]:
def rician_wholedataset(dataset, BaseVal, Scale, NumSamples):
    noise = np.random.normal(scale=Scale, size=(
            NumSamples, 2)) + [[BaseVal, 0]]
    noise = np.linalg.norm(noise, axis=1)
    listd = [[dataset[i], noise] for i in  list(range(dataset.shape[0]))] 
    try:
        cpu_count = multiprocessing.cpu_count()
        pool = multiprocessing.Pool(processes = cpu_count) 
        data = pool.starmap(ricianprocess, listd)
        data = np.stack(data)
    finally:
        pool.close()
        pool.join()
        
    return data


In [18]:
# Grab Currrent Time Before Running the Code
start = time.time()
a = rician_wholedataset_inplace(np.zeros((1000,240,240,1)), 5, 3, 240*240)
# Grab Currrent Time After Running the Code
end = time.time()

#Subtract Start Time from The End Time
total_time = end - start
print(total_time)
# a = rician_wholedataset(t1ce_data, 5, 3, 240*240)


/tmp/ipykernel_179903/1039886119.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  data[i, :, :, :] *= 255.0/data[i, :, :, :].max()
/tmp/ipykernel_179903/1039886119.py:5: RuntimeWarning: invalid value encountered in multiply
  data[i, :, :, :] *= 255.0/data[i, :, :, :].max()


0.6260519027709961


In [170]:
a.shape

(1000, 240, 240, 1)

In [52]:
def rayleigh_wholedataset(data,rand_scale=8,NumSamples=240):
    for i in range(data.shape[0]):
        raynoise = np.random.rayleigh(rand_scale, NumSamples*NumSamples)
        raynoise = raynoise.reshape(NumSamples, NumSamples)
        data[i, :, :, :] = img_as_float(data[i, :, :, :]*255) + raynoise.reshape(data[i, :, :, :].shape)
        data[i, :, :, :] *= 255.0/data[i, :, :, :].max()
        data[i] = data[i] / 255.0
    return data

In [53]:
a = rayleigh_wholedataset(np.zeros((1000,240,240,1)))

In [54]:
start = time.time()
a = rayleigh_wholedataset(np.zeros((1000,240,240,1)))
# Grab Currrent Time After Running the Code
end = time.time()

#Subtract Start Time from The End Time
total_time = end - start
print(total_time)

2.4239661693573


In [55]:
from skimage.metrics import peak_signal_noise_ratio

In [144]:
def psnr_wholedataset(dataset_original, dataset_denoised):

    avgpsnr = 0
    for i in range(dataset_original.shape[0]):
        true_min, true_max = np.min(dataset_original[i, :, :, :]), np.max(dataset_original[i, :, :, :])
        dataRange = abs(true_min)+abs(true_max)
        psnr = peak_signal_noise_ratio(dataset_original[i], dataset_denoised[i], data_range=dataRange)
        avgpsnr += psnr
        avgpsnr = avgpsnr/2

    return avgpsnr

In [ ]:
def worker_wrapper(arg):
    args = arg[0:2]
    kwargs = arg[3]
    return

In [150]:
from itertools import repeat

def starmap_with_kwargs(pool, fn, args_iter, kwargs_iter):
    args_for_starmap = zip(repeat(fn), args_iter, kwargs_iter)
    return pool.starmap(apply_args_and_kwargs, args_for_starmap)

def apply_args_and_kwargs(fn, args, kwargs):
    return fn(*args, **kwargs)





def psnr_wholedataset_pool(dataset_original, dataset_denoised):
    true_min, true_max = np.min(dataset_original[i, :, :, :]), np.max(dataset_original[i, :, :, :])
    dataRange = abs(true_min)+abs(true_max)
    # args_iter = zip(list(unstack(dataset_original,0)), list(unstack(dataset_denoised,0)))
    args_iter = zip(list(unstack(dataset_original,0)), list(unstack(dataset_denoised,0)))   
    try:
        cpu_count = multiprocessing.cpu_count()
        pool = multiprocessing.Pool(processes = cpu_count) 
        kwargs_iter = repeat(dict(data_range=dataRange))
        branches = starmap_with_kwargs(pool, peak_signal_noise_ratio , args_iter, kwargs_iter)
    finally:
        pool.close()
        pool.join()

    return sum(branches)/dataset_original.shape[0]


def psnr_wholedataset_pool1(data):
    true_min, true_max = np.min(dataset_original[i, :, :, :]), np.max(dataset_original[i, :, :, :])
    dataRange = abs(true_min)+abs(true_max)
    args_iter = data 
    try:
        cpu_count = multiprocessing.cpu_count()
        pool = multiprocessing.Pool(processes = cpu_count) 
        kwargs_iter = repeat(dict(data_range=dataRange))
        branches = starmap_with_kwargs(pool, peak_signal_noise_ratio , args_iter, kwargs_iter)
    finally:
        pool.close()
        pool.join()

    return sum(branches)/dataset_original.shape[0]

In [151]:
def unstack(a, axis=0):
    return np.moveaxis(a, axis, 0)

In [152]:
start = time.time()
a = psnr_wholedataset_pool(t1ce_data, utils.rician_wholedataset(t1ce_data, 5,3,240*240))
# Grab Currrent Time After Running the Code
end = time.time()

#Subtract Start Time from The End Time
total_time = end - start
print(total_time)

start = time.time()
a = psnr_wholedataset(t1ce_data, utils.rician_wholedataset(t1ce_data, 5,3,240*240))
# Grab Currrent Time After Running the Code
end = time.time()

#Subtract Start Time from The End Time
total_time = end - start
print(total_time)

dataset_original = t1ce_data
dataset_denoised = utils.rician_wholedataset(t1ce_data, 5,3,240*240)
data = zip(list(unstack(dataset_original,0)), list(unstack(dataset_denoised,0)))
start = time.time()
a = psnr_wholedataset_pool1(data)
# Grab Currrent Time After Running the Code
end = time.time()

#Subtract Start Time from The End Time
total_time = end - start
print(total_time)



1.3364076614379883
0.9268815517425537
0.5205528736114502


In [141]:
d = np.zeros((100,256,256,1))
a = np.hsplit(d)

TypeError: _hvdsplit_dispatcher() missing 1 required positional argument: 'indices_or_sections'

In [71]:
a.shape

(256, 256, 100)

In [81]:
list(unstack(d, 0)

100

In [85]:
for i,j in zip([1,2,3],[4,5,6]):
    print(i,j)

1 4
2 5
3 6


In [ ]:
def rician_wholedataset(dataset, BaseVal, Scale, NumSamples):
    noise = np.random.normal(scale=Scale, size=(
            NumSamples, 2)) + [[BaseVal, 0]]
    noise = np.linalg.norm(noise, axis=1)
    listd = [[dataset[i], noise] for i in  list(range(dataset.shape[0]))] 
    try:
        cpu_count = multiprocessing.cpu_count()
        pool = multiprocessing.Pool(processes = cpu_count) 
        data = pool.starmap(ricianprocess, listd)
        data = np.stack(data)
    finally:
        pool.close()
        pool.join()
        
    return data


In [106]:
**kwargs = dict(das= 1)
args , kwargs = (1,kwargs)


SyntaxError: invalid syntax (2946256202.py, line 1)

In [111]:
from multiprocessing import Pool

In [112]:
cpu_count = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes = cpu_count) 

In [116]:
from itertools import repeat

def starmap_with_kwargs(pool, fn, args_iter, kwargs_iter):
    args_for_starmap = zip(repeat(fn), args_iter, kwargs_iter)
    return pool.starmap(apply_args_and_kwargs, args_for_starmap)

def apply_args_and_kwargs(fn, args, kwargs):
    return fn(*args, **kwargs)


args_iter = zip(list(unstack(t1ce_data,0)), list(unstack(t1ce_data,0)))
kwargs_iter = repeat(dict(data_range=255))
branches = starmap_with_kwargs(pool, peak_signal_noise_ratio , args_iter, kwargs_iter)


In [154]:
zip([2,3],[3,4])

TypeError: 'zip' object is not subscriptable